<a href="https://colab.research.google.com/github/jespimentel/tce-despesas-municipais/blob/main/TCE_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análise de dados do TCE-SP

## Contexto

O Tribunal de Contas do Estado de São Paulo disponibiliza APIs de consulta pública com informações detalhadas das receitas e despesas mensais dos Municípios jurisdicionados.

O programa consulta as despesas do período de um ano do Município selecionado pelo usuário. Reúne os dados num único dataframe e prossegue com as análises entendidas como relevantes.

## API

APIs (acrônimo de *Application Programming Interface*) são interfaces que facilitam a integração de programas. Possuem requisitos e protolocos que permitem a comunicação entre aplicações ou sistemas. Entre sistemas web, as APIs se comunicam, normalmente, nos padrões JSON (mais comum) e XML. 

Quando consumimos uma API como a do TCE, trazemos os dados para o nosso programa sem nos preocupar com a forma como tais informações são processadas na origem.

## Análises

Este Jupyter Notebook cria a requisição ao TCE com o nome do Município, ano e mês. Um loop repete a operação para os meses anteriores, até completar o período de um ano. Os arquivos JSON de resposta geram dataframes, que vão sendo concatenados para formar um dataframe único.

São identificados:

### Pessoas físicas:

* Os maiores pagamentos individuais realizados a PF
* Os maiores pagamentos acumulados feitos a PF
* As maiores quantidades de pagamentos realizados a cada PF
* Informações estatísticas sobre pagamentos feitos a PF

### Pessoas jurídicas:

* Os maiores pagamentos individuais realizados a PJ
* Os maiores pagamentos acumulados feitos a PJ
* As maiores quantidades de pagamentos realizados a cada PJ
* Informações estatísticas sobre pagamentos feitos a PJ


## Gravação de arquivo Excel

Ao final da análise, um arquivo Excel é gerado com os dados organizados.

# Criação do DataFrame de Despesas

In [1]:
# Bibliotecas

import json
import requests
import pandas as pd

In [2]:
# Data

from datetime import date
hoje = date.today()
ano_corrente = hoje.year
mes_corrente = hoje.month
print (f'Hoje é {hoje.strftime("%d/%m/%Y")}')

Hoje é 27/06/2022


In [ ]:
# Relação de municípios

path_municipios = 'https://transparencia.tce.sp.gov.br/api/json/municipios'

municipios = requests.get(path_municipios)
if municipios.status_code == 200:
    municipios_data = json.loads(municipios.content)
else:
  print(f'Erro na requisição para {path_municipios}')

print (f'Total de municípios: {len(municipios_data)}\n')

n = 0
for item in municipios_data:
  n +=1
  print (item['municipio'], end = '; ')
  if n%6 == 0:
    print ()

In [4]:
# Seleção do município
municipio_dict = dict()

for item in range(len(municipios_data)):
  municipio_dict[municipios_data[item]['municipio']] = municipios_data[item]['municipio_extenso']

municipio = input('Entre com o município: ')
print(f'Você escolheu: {municipio_dict[municipio]}')

Entre com o município: piracicaba
Você escolheu: Piracicaba


In [5]:
# Função

def encontra_dados_mais_atuais(municipio, ano, mes):
  '''Encontra os dados mais atuais testando os requests'''
  for m in range (int(mes), 0, -1):
    mes = str(m)
    teste_requests = requests.get(f'https://transparencia.tce.sp.gov.br/api/json/despesas/{municipio}/{ano}/{mes}')
    despesas_data = json.loads(teste_requests.content)
    if len(despesas_data)!= 0:
      print (f'Dados mais atuais: {ano}/{mes}')
      return (municipio, ano, mes)
  ano = str(int(ano) - 1)
  encontra_dados_mais_atuais(municipio, ano, '12') 

In [6]:
# Dataframe de despesas de 1 ano do Município escolhido

colunas = ['orgao', 'mes', 'evento', 'nr_empenho', 'id_fornecedor', 'nm_fornecedor', 'dt_emissao_despesa', 'vl_despesa']
df_despesas = pd.DataFrame(columns=colunas) # Criação do dataframe vazio

dados_mais_atuais = encontra_dados_mais_atuais(municipio, str(ano_corrente), str(mes_corrente))  # ('piracicaba', '2022', '4')

# Ano corrente
for m in range (int(dados_mais_atuais[2]), 0, -1):
  mes = str(m)
  ano = dados_mais_atuais[1]
  path_despesas = f'https://transparencia.tce.sp.gov.br/api/json/despesas/{municipio}/{ano}/{mes}'
  despesas = requests.get(path_despesas)
  if despesas.status_code == 200:
    despesas_data = json.loads(despesas.content)
    df_parcial = pd.DataFrame.from_dict(despesas_data)
    df_despesas = pd.concat([df_despesas, df_parcial])
  else:
     print(f'Erro na requisição para {path_despesas}')

# Ano anterior
for m in range (12, int(dados_mais_atuais[2]), -1):
  mes = str(m)
  ano = str (int(dados_mais_atuais[1]) - 1)
  path_despesas = f'https://transparencia.tce.sp.gov.br/api/json/despesas/{municipio}/{ano}/{mes}'
  despesas = requests.get(path_despesas)
  if despesas.status_code == 200:
    despesas_data = json.loads(despesas.content)
    df_parcial = pd.DataFrame.from_dict(despesas_data)
    df_despesas = pd.concat([df_despesas, df_parcial])
  else:
     print(f'Erro na requisição para {path_despesas}')

Dados mais atuais: 2022/4


In [7]:
# Tratamento dos dados

df_despesas['dt_emissao_despesa'] = pd.to_datetime(df_despesas['dt_emissao_despesa'], dayfirst=True)
df_despesas['vl_despesa'] = df_despesas['vl_despesa'].str.replace(',', '.')
df_despesas['vl_despesa'] = df_despesas['vl_despesa'].astype('float')
df_despesas['pessoa_fisica_?'] = df_despesas['id_fornecedor'].str.contains('PESSOA FÍSICA')

In [8]:
# Sample do Dataframe

df_despesas.sample(5)

,orgao,mes,evento,nr_empenho,id_fornecedor,nm_fornecedor,dt_emissao_despesa,vl_despesa,pessoa_fisica_?
2917,PREFEITURA MUNICIPAL DE PIRACICABA,Fevereiro,Valor Pago,00037-2022,CNPJ - PESSOA JURÍDICA - 54408778000163,FREIOTEC COM.DE LONAS P/FREIOS E FRICCAO LTDA,2022-02-22,848.40,False
3066,IPASP - CNPJ 51.327.724-0001-85,Janeiro,Valor Pago,00001-2022,CNPJ - PESSOA JURÍDICA - 04088208000165,CGMP-CENTRO GESTAO DE MEIOS DE PAG. S/A,2022-01-10,85.22,False
4866,PREFEITURA MUNICIPAL DE PIRACICABA,Setembro,Valor Liquidado,00007-2021,CNPJ - PESSOA JURÍDICA - 03367523000169,ASSOC.DE PAIS E AMIGOS AUTISTAS DE PIRAC.AUMA,2021-09-23,8281.00,False
3044,IPPLAP - CNPJ 05.859.181-0001-20,Janeiro,Valor Pago,00010-2022,CNPJ - PESSOA JURÍDICA - 03772079000167,RODRIGO E CIA COM.E LOC.DE MAT.PARA INF.LTDA,2022-01-14,1000.00,False
6177,FUNDACAO MUNICIPAL DE ENSINO,Dezembro,Valor Pago,01350-2021,CNPJ - PESSOA JURÍDICA - 00360305033207,CAIXA ECONOMICA FEDERAL,2021-12-21,452.25,False


# Pessoa física

In [9]:
# Top 20 pagamentos individuais a pessoas físicas

criterio = (df_despesas['evento'] == 'Valor Pago') & (df_despesas['pessoa_fisica_?']==True)
pf_pagamentos_individuais = df_despesas[criterio].sort_values(by=['vl_despesa'], ascending=False)
pf_pagamentos_individuais.head(20)

,orgao,mes,evento,nr_empenho,id_fornecedor,nm_fornecedor,dt_emissao_despesa,vl_despesa,pessoa_fisica_?
2836,PREFEITURA MUNICIPAL DE PIRACICABA,Julho,Valor Pago,00865-2021,PESSOA FÍSICA - 008606,MOISES FRANCISCO BALDO TAGLIETTA,2021-07-30,123275.89,True
1574,PREFEITURA MUNICIPAL DE PIRACICABA,Julho,Valor Pago,00224-2021,PESSOA FÍSICA - 852868,LASARO LUIS NEGRI,2021-07-08,95423.40,True
2770,PREFEITURA MUNICIPAL DE PIRACICABA,Novembro,Valor Pago,01206-2021,PESSOA FÍSICA - 948628,ELISABETH ZAMBOM,2021-11-05,92783.79,True
1907,PREFEITURA MUNICIPAL DE PIRACICABA,Abril,Valor Pago,00038-2022,PESSOA FÍSICA - 354508,JOSÉ ANTÔNIO PERIN,2022-04-08,90286.96,True
3983,SEMAE - CNPJ 50.853.555-0001-54,Março,Valor Pago,540-2022,PESSOA FÍSICA - 042828,ELAINE CONTIERO RIBEIRO,2022-03-16,87918.29,True
3977,PREFEITURA MUNICIPAL DE PIRACICABA,Dezembro,Valor Pago,01379-2021,PESSOA FÍSICA - 613237,EDSON LUIS DE CAMPOS BICUDO,2021-12-28,76464.44,True
3683,PREFEITURA MUNICIPAL DE PIRACICABA,Abril,Valor Pago,00134-2022,PESSOA FÍSICA - 674328,FELIPE FISCHER IGREJA,2022-04-08,75111.58,True
1947,PREFEITURA MUNICIPAL DE PIRACICABA,Junho,Valor Pago,00662-2021,PESSOA FÍSICA - 261108,PAULO CESAR GAIOTTO,2021-06-04,68927.46,True
2291,PREFEITURA MUNICIPAL DE PIRACICABA,Outubro,Valor Pago,01159-2021,PESSOA FÍSICA - 438168,VALQUIRIA BATISTA SACHS,2021-10-29,68125.46,True
2450,PREFEITURA MUNICIPAL DE PIRACICABA,Outubro,Valor Pago,00352-2021,PESSOA FÍSICA - 307898,EDIMILSON OLIVEIRA SANTOS,2021-10-22,67691.64,True


In [10]:
# Top 20 pagamentos acumulados PF

criterio = (df_despesas['evento'] == 'Valor Pago') & (df_despesas['pessoa_fisica_?']==True)
pf_pagamentos_acumulados = df_despesas[criterio].groupby(['id_fornecedor', 'nm_fornecedor'])['vl_despesa'].sum().sort_values(ascending=False)
pf_pagamentos_acumulados.head(20)

id_fornecedor           nm_fornecedor                   
PESSOA FÍSICA - 218758  LIRIAM YUKIE YAMAMOTO               277658.99
PESSOA FÍSICA - 331438  BERNADETE DE LOURDES NUNES PAIS     213576.08
PESSOA FÍSICA - 302718  GETULIO FERNANDES DO ROSARIO        182763.75
PESSOA FÍSICA - 736688  DONIZETE APARECIDO OLIVEIRA         165000.70
PESSOA FÍSICA - 179248  ROBERTO DA SILVA FERREIRA           151145.96
PESSOA FÍSICA - 610412  JULIANA PEREIRA NERY                144000.00
PESSOA FÍSICA - 008606  MOISES FRANCISCO BALDO TAGLIETTA    129247.02
PESSOA FÍSICA - 852868  LASARO LUIS NEGRI                   119279.25
PESSOA FÍSICA - 599938  CLAUDIO LUIZ BRAZ BORTOLETTO        100098.12
PESSOA FÍSICA - 674328  FELIPE FISCHER IGREJA                98883.02
PESSOA FÍSICA - 948628  ELISABETH ZAMBOM                     92783.79
PESSOA FÍSICA - 327758  DIRCE TREVISAN                       91782.38
PESSOA FÍSICA - 354508  JOSÉ ANTÔNIO PERIN                   90286.96
PESSOA FÍSICA - 234678  JOSE ANTO

In [11]:
# Top 20 quantidades de pagamentos realizados a PF

criterio = (df_despesas['evento'] == 'Valor Pago') & (df_despesas['pessoa_fisica_?']==True)
quant_pagamentos_pf = df_despesas[criterio].groupby(['id_fornecedor', 'nm_fornecedor'])['vl_despesa'].count().sort_values(ascending=False)
quant_pagamentos_pf.head(20)

id_fornecedor           nm_fornecedor                     
PESSOA FÍSICA - 179248  ROBERTO DA SILVA FERREIRA             126
PESSOA FÍSICA - 387178  FABIANA APARECIDA MARTINS DA SILVA     52
PESSOA FÍSICA - 736688  DONIZETE APARECIDO OLIVEIRA            51
PESSOA FÍSICA - 546368  MAURICIO BOSCARIOL GUARDIA             50
PESSOA FÍSICA - 302718  GETULIO FERNANDES DO ROSARIO           49
PESSOA FÍSICA - 171618  MARIA JOELMA DA ROSA                   46
PESSOA FÍSICA - 278748  MARIA LUCINEI XAVIER                   44
PESSOA FÍSICA - 507735  FLAVIA DIAS SOARES DE SA               44
PESSOA FÍSICA - 137228  IVAN JOSE ZOTELLI                      44
PESSOA FÍSICA - 965368  THALITA SCHIABEL TARARAM               44
PESSOA FÍSICA - 733468  ALVARO JOSE POLISEL                    42
PESSOA FÍSICA - 828428  DAYANE GABRIELE BORTOLETO              41
PESSOA FÍSICA - 406398  VANESSA DE CASSIA DINIZ MENEGON        40
PESSOA FÍSICA - 435868  LUIS CARLOS RUBIA                      38
PESSOA FÍSICA - 2

In [12]:
# Top 20 Desvios-padrão do pagamento de PF

criterio = (df_despesas['evento'] == 'Valor Pago') & (df_despesas['pessoa_fisica_?']==True)
pf_std = df_despesas[criterio].groupby(['id_fornecedor', 'nm_fornecedor'])['vl_despesa'].describe().sort_values(by='std', ascending=False)
pf_std.head(20)

,,count,mean,std,min,25%,50%,75%,max
id_fornecedor,nm_fornecedor,,,,,,,,
PESSOA FÍSICA - 008606,MOISES FRANCISCO BALDO TAGLIETTA,2.0,64623.510000,82946.991261,5971.13,35297.3200,64623.510,93949.7000,123275.89
PESSOA FÍSICA - 042828,ELAINE CONTIERO RIBEIRO,4.0,8782.080000,58823.244214,-54361.87,-13911.5425,785.950,23479.5725,87918.29
PESSOA FÍSICA - 852868,LASARO LUIS NEGRI,2.0,59639.625000,50605.899918,23855.85,41747.7375,59639.625,77531.5125,95423.40
PESSOA FÍSICA - 438168,VALQUIRIA BATISTA SACHS,2.0,36569.390000,44627.022169,5013.32,20791.3550,36569.390,52347.4250,68125.46
PESSOA FÍSICA - 868498,LUIZ FREDERICO SABLEWSKI GRAU,2.0,34683.735000,43597.170640,3855.88,19269.8075,34683.735,50097.6625,65511.59
PESSOA FÍSICA - 674328,FELIPE FISCHER IGREJA,2.0,49441.510000,36302.961141,23771.44,36606.4750,49441.510,62276.5450,75111.58
PESSOA FÍSICA - 994018,ANA MARIA DE SANTIS,2.0,26159.075000,34062.825645,2073.02,14116.0475,26159.075,38202.1025,50245.13
PESSOA FÍSICA - 930828,SILVANA MARIA MAGRO KOREN,4.0,9128.137500,33035.794979,-34660.02,-7275.6075,18256.275,34660.0200,34660.02
PESSOA FÍSICA - 017008,OSVALDO BENEDITO RODRIGUES,2.0,28378.665000,32685.763473,5266.34,16822.5025,28378.665,39934.8275,51490.99


# Pessoa jurídica

In [13]:
# Top 20 pagamentos individuais a pessoas jurídicas

criterio = (df_despesas['evento'] == 'Valor Pago') & (df_despesas['pessoa_fisica_?']==False)
pj_pagamentos_individuais = df_despesas[criterio].sort_values(by=['vl_despesa'], ascending=False)
pj_pagamentos_individuais.head(20)

,orgao,mes,evento,nr_empenho,id_fornecedor,nm_fornecedor,dt_emissao_despesa,vl_despesa,pessoa_fisica_?
1643,PREFEITURA MUNICIPAL DE PIRACICABA,Abril,Valor Pago,00036-2022,CNPJ - PESSOA JURÍDICA - 15664292000134,PIRACICABA AMBIENTAL SOCIEDADE ANONIMA,2022-04-08,10221253.49,False
2041,PREFEITURA MUNICIPAL DE PIRACICABA,Março,Valor Pago,00036-2022,CNPJ - PESSOA JURÍDICA - 15664292000134,PIRACICABA AMBIENTAL SOCIEDADE ANONIMA,2022-03-11,8525219.71,False
3246,SEMAE - CNPJ 50.853.555-0001-54,Fevereiro,Valor Pago,150-2022,CNPJ - PESSOA JURÍDICA - 15384637000104,ÁGUAS DO MIRANTE S.A.,2022-02-15,8465176.83,False
5951,SEMAE - CNPJ 50.853.555-0001-54,Abril,Valor Pago,150-2022,CNPJ - PESSOA JURÍDICA - 15384637000104,ÁGUAS DO MIRANTE S.A.,2022-04-19,8372725.41,False
2901,PREFEITURA MUNICIPAL DE PIRACICABA,Fevereiro,Valor Pago,00036-2022,CNPJ - PESSOA JURÍDICA - 15664292000134,PIRACICABA AMBIENTAL SOCIEDADE ANONIMA,2022-02-08,8341133.62,False
4126,SEMAE - CNPJ 50.853.555-0001-54,Outubro,Valor Pago,121-2021,CNPJ - PESSOA JURÍDICA - 15384637000104,ÁGUAS DO MIRANTE S.A.,2021-10-22,8292985.34,False
4002,SEMAE - CNPJ 50.853.555-0001-54,Março,Valor Pago,150-2022,CNPJ - PESSOA JURÍDICA - 15384637000104,ÁGUAS DO MIRANTE S.A.,2022-03-17,8090917.71,False
3439,SEMAE - CNPJ 50.853.555-0001-54,Setembro,Valor Pago,121-2021,CNPJ - PESSOA JURÍDICA - 15384637000104,ÁGUAS DO MIRANTE S.A.,2021-09-23,8040329.08,False
5964,SEMAE - CNPJ 50.853.555-0001-54,Dezembro,Valor Pago,2051-2021,CNPJ - PESSOA JURÍDICA - 15384637000104,ÁGUAS DO MIRANTE S.A.,2021-12-23,8039814.80,False
4077,SEMAE - CNPJ 50.853.555-0001-54,Junho,Valor Pago,121-2021,CNPJ - PESSOA JURÍDICA - 15384637000104,ÁGUAS DO MIRANTE S.A.,2021-06-22,7845117.65,False


In [14]:
# Top 20 pagamentos acumulados a PJ

criterio = (df_despesas['evento'] == 'Valor Pago') & (df_despesas['pessoa_fisica_?']==False)
pj_pagamentos_acumulados = df_despesas[criterio].groupby(['id_fornecedor', 'nm_fornecedor'])['vl_despesa'].sum().sort_values(ascending=False)
pj_pagamentos_acumulados.head(20)

id_fornecedor                                   nm_fornecedor                                                                             
INSCRIÇÃO GENÉRICA-OUTROS - 3000036             FOLHA DE PAGAMENTO DE PESSOAL                                                                 4.951513e+08
CNPJ - PESSOA JURÍDICA - 46341038000129         PREFEITURA DO MUNIC. DE PIRACICABA                                                            1.092523e+08
CNPJ - PESSOA JURÍDICA - 15384637000104         ÁGUAS DO MIRANTE S.A.                                                                         9.437501e+07
CNPJ - PESSOA JURÍDICA - 15664292000134         PIRACICABA AMBIENTAL SOCIEDADE ANONIMA                                                        8.899245e+07
CNPJ - PESSOA JURÍDICA - 54370630000187         IRMANDADE DA STA CASA DE MISERIC.DE PIRACICAB                                                 7.497062e+07
CNPJ - PESSOA JURÍDICA - 51327724000185         INST.PREV.ASSIT.SOC.FUN.PUBL.PIRACICAB

In [15]:
# Top 20 quantidades de pagamentos realizados a PJ

criterio = (df_despesas['evento'] == 'Valor Pago') & (df_despesas['pessoa_fisica_?']==False)
quant_pagamentos_pj = df_despesas[criterio].groupby(['id_fornecedor', 'nm_fornecedor'])['vl_despesa'].count().sort_values(ascending=False)
quant_pagamentos_pj.head(20)

id_fornecedor                                   nm_fornecedor                                                                             
INSCRIÇÃO GENÉRICA-OUTROS - 3000036             FOLHA DE PAGAMENTO DE PESSOAL                                                                 1768
CNPJ - PESSOA JURÍDICA - 29979036035793         INSS-INST.NAC.SEG.SOCIAL                                                                       504
CNPJ - PESSOA JURÍDICA - 00360305033207         CAIXA ECONOMICA FEDERAL                                                                        491
CNPJ - PESSOA JURÍDICA - 14144192000114         D.P. QUARTAROLO GERENCIAMENTO DE FROTAS EIREL                                                  439
CNPJ - PESSOA JURÍDICA - 51327724000185         INST.PREV.ASSIT.SOC.FUN.PUBL.PIRACICABA                                                        424
CNPJ - PESSOA JURÍDICA - 46341038000129         PREFEITURA DO MUNIC. DE PIRACICABA                                            

In [16]:
# Top 20 Desvios-padrão do pagamento de PJ

criterio = (df_despesas['evento'] == 'Valor Pago') & (df_despesas['pessoa_fisica_?']==False)
pj_std = df_despesas[criterio].groupby(['id_fornecedor', 'nm_fornecedor'])['vl_despesa'].describe().sort_values(by='std', ascending=False)
pj_std.head(20)

,,count,mean,std,min,25%,50%,75%,max
id_fornecedor,nm_fornecedor,,,,,,,,
CNPJ - PESSOA JURÍDICA - 15664292000134,PIRACICABA AMBIENTAL SOCIEDADE ANONIMA,23.0,3.869237e+06,3.023107e+06,883880.88,1.159507e+06,2280000.000,5.952647e+06,10221253.49
CNPJ - PESSOA JURÍDICA - 15384637000104,ÁGUAS DO MIRANTE S.A.,14.0,6.741072e+06,2.268133e+06,2500000.00,6.180772e+06,7824480.535,8.078271e+06,8465176.83
CNPJ - PESSOA JURÍDICA - 08404654000192,CESAR CONTAINERS E EQUIPAMENTOS EIRELI,4.0,1.337673e+06,1.524573e+06,122323.59,2.339186e+05,906523.120,2.010278e+06,3415322.78
CNPJ - PESSOA JURÍDICA - 54370630000187,IRMANDADE DA STA CASA DE MISERIC.DE PIRACICAB,47.0,1.595119e+06,1.469814e+06,142.42,2.687000e+05,1644627.000,2.721639e+06,4197153.58
CNPJ - PESSOA JURÍDICA - 54384631000261,ASSOC.DOS FORN.DE CANA DE PIRACICABA,67.0,8.652101e+05,9.144959e+05,0.00,9.725000e+04,341700.000,1.445656e+06,3039611.68
INSCRIÇÃO GENÉRICA-OUTROS - 3000036,FOLHA DE PAGAMENTO DE PESSOAL,1768.0,2.800629e+05,7.136859e+05,0.00,6.724740e+03,34718.910,2.015547e+05,5654600.71
CNPJ - PESSOA JURÍDICA - 46341038000129,PREFEITURA DO MUNIC. DE PIRACICABA,413.0,2.645333e+05,6.478454e+05,36.52,6.419820e+03,37818.680,2.358320e+05,5191203.14
CNPJ - PESSOA JURÍDICA - 51327724000185,INST.PREV.ASSIT.SOC.FUN.PUBL.PIRACICABA,424.0,1.540443e+05,5.088164e+05,0.01,1.711090e+03,9845.600,5.863671e+04,4716738.67
INSCRIÇÃO GENÉRICA-OUTROS - 4000001,RETENCAO E PAGAMENTO AO PASEP,17.0,4.990741e+05,4.505998e+05,65852.00,8.631048e+04,135708.100,9.351780e+05,1092718.82


In [17]:
# Criação do arquivo Excel

with pd.ExcelWriter(f'{municipio}_{hoje.strftime("%Y_%m")}.xlsx') as writer: 
  pf_pagamentos_individuais.to_excel(writer, sheet_name='PF - pagamentos individuais')
  pf_pagamentos_acumulados.to_excel(writer, sheet_name='PF - pagamentos acumulados')
  quant_pagamentos_pf.to_excel(writer, sheet_name='PF - quantidades de pagamentos')
  pf_std.to_excel(writer, sheet_name='PF - estatística')
  pj_pagamentos_individuais.to_excel(writer, sheet_name='PJ - pagamentos individuais')
  pj_pagamentos_acumulados.to_excel(writer, sheet_name='PJ - pagamentos acumulados')
  quant_pagamentos_pj.to_excel(writer, sheet_name='PJ - quantidades de pagamentos')
  pj_std.to_excel(writer, sheet_name='PJ - estatística')
  df_despesas.to_excel(writer, sheet_name='Todas as despesas do período')

# Outros ensaios


In [18]:
pf_std[pf_std['mean']>20000]

,,count,mean,std,min,25%,50%,75%,max
id_fornecedor,nm_fornecedor,,,,,,,,
PESSOA FÍSICA - 008606,MOISES FRANCISCO BALDO TAGLIETTA,2.0,64623.510,82946.991261,5971.13,35297.3200,64623.510,93949.7000,123275.89
PESSOA FÍSICA - 852868,LASARO LUIS NEGRI,2.0,59639.625,50605.899918,23855.85,41747.7375,59639.625,77531.5125,95423.40
PESSOA FÍSICA - 438168,VALQUIRIA BATISTA SACHS,2.0,36569.390,44627.022169,5013.32,20791.3550,36569.390,52347.4250,68125.46
PESSOA FÍSICA - 868498,LUIZ FREDERICO SABLEWSKI GRAU,2.0,34683.735,43597.170640,3855.88,19269.8075,34683.735,50097.6625,65511.59
PESSOA FÍSICA - 674328,FELIPE FISCHER IGREJA,2.0,49441.510,36302.961141,23771.44,36606.4750,49441.510,62276.5450,75111.58
...,...,...,...,...,...,...,...,...,...
PESSOA FÍSICA - 964208,ALESSANDRA BENIANA CAPISTRANO MARTINS,1.0,22702.660,NaN,22702.66,22702.6600,22702.660,22702.6600,22702.66
PESSOA FÍSICA - 979948,MARIA IZABEL PERES DA CRUZ,1.0,27080.870,NaN,27080.87,27080.8700,27080.870,27080.8700,27080.87
PESSOA FÍSICA - 986838,VANESSA MORAES ABDALA GUARDA,1.0,30472.080,NaN,30472.08,30472.0800,30472.080,30472.0800,30472.08
